# midiAI

In this notebook, I'll go through my process of using `music21` to create lists of information about songs generated from the `.midi` files in `data/` generated from the [*OpenBook*](https://veltzer.github.io/openbook/) repository.

I'll start by importing my libraries first:

- `music21`: I'll be using `converter` to parse the midi file, `interval` and `key` to change the key to C major, and `note` to get the note data
- `Path`: Just using `pathlib` for my files
- `tqdm`: I'll use `tqdm.noetbook.tqdm` to show the progress of converting the `.midi` files to `str`ings

In [130]:
from music21 import converter, interval, key, note
from pathlib import Path
from pprint import pprint
from tqdm.notebook import tqdm

## `data/` filepath

The first step is to define my `data_path` (which contains all of my `.midi` files) and create a list of each file in the directory.

In [131]:
data_path = Path("data")
files = [f for f in data_path.iterdir() if f.is_file() and f.suffix.lower() == ".midi"]

## Parsing with `music21`

Next, I'll define my functions to return a string of midi notes and lengths.

In [132]:
def parse(fp):
	s = converter.parse(fp)
	o_key = s.flatten().getElementsByClass(key.KeySignature)[0]
	if (o_key.tonic == "C" and o_key.mode == "major") or (o_key.tonic == "A" and o_key.mode == "minor"):
		return s
	t_key = key.Key("C") if o_key.mode == "major" else key.Key("A", "minor")
	i = interval.Interval(o_key.tonic, t_key.tonic)
	ns = s.transpose(i)
	return ns

def get_notes(s):
	song_data = []
	for p in s.parts[1]:
		for n in p:
			n_name = n.pitch.midi if isinstance(n, note.Note) else -1
			n_len = round(float(n.quarterLength), 2)
			n_info = f"{str(n_name)}:{str(n_len)}"
			if n_len != 0.0:
				song_data.append(n_info)
	return song_data

def song_to_str(n):
	return f"{' '.join(n)}"

def midi_to_str(fp):
	return song_to_str(get_notes(parse(fp)))

## Processing `.midi` files

Let's put them into use by creating a list of each song's parsed string.

In [133]:
songs = []

for fp in tqdm(files, unit="file"):
  song_str = song_to_str(get_notes(parse(fp)))
  songs.append(song_str)

  0%|          | 0/153 [00:00<?, ?file/s]

Next, I'll create a text file `prepared_notes.txt` which contains each of the strings separated by two newline characters.

In [134]:
all_songs = "\n\n".join(songs)
with open("prepared_notes.txt", "w", encoding="utf-8") as f:
  f.write(all_songs)

## Examining `prepared_notes.txt`

Let's take a look at some information in our prepared file.

In [135]:
songs = open("prepared_notes.txt", "r", encoding="utf-8").read().strip()
notes = songs.split()

note_names = set()
note_lens = set()

for n in notes:
  note_name = int(n.split(":")[0])
  note_len = float(n.split(":")[1])
  note_names.add(note_name)
  if round(note_len % 1, 2) in [0, 0.25, 0.33, 0.5, 0.67, 0.75]:
    note_lens.add(float(n.split(":")[1]))

note_names = sorted(list(note_names))
note_lens = sorted(list(note_lens))

Now let's get a couple of dictionaries to map the midi note values to their associated names.

In [136]:
midi_numbers = [i for i in range(43, 89)]

note_names = ["C", "C#", "D", "E-", "E", "F", "F#", "G", "G#", "A", "B-", "B"]

midi_value_to_note = {n: f"{note_names[n % 12]}{(n // 12) - 1}" for n in midi_numbers}

midi_value_to_note[-1] = "REST"

In [137]:
readable_notes = []
for n in notes:
  note_name = int(n.split(":")[0])
  note_len = float(n.split(":")[1])
  if round(note_len % 1, 2) in [0, 0.25, 0.33, 0.5, 0.67, 0.75]:
    readable_notes.append((note_name, note_len))
readable_notes = sorted(readable_notes, key=lambda x: x[0])

new_notes = []

for n in readable_notes:
  v = str(n[0])
  l = str(n[1])
  new_notes.append(":".join([v,l]))